# Werkzeuge zur Darstellung von Hystersekurven

Mit den folgenden Code-Fragmenten zeigen wir Ihnen: 

 * Wie man *csv*-Datei mit Hilfe von PhyPraKit oder *pandas* **einließt**.
 * Wie man die eingelesenen Daten auf ein handhabbares Maß **reduziert**.
 * Wie man die Daten mit Hilfe von [*Spline*](https://de.wikipedia.org/wiki/Spline)-Funktionen **interpoliert**.
 * Wie man die angepassten *Spline*-Funktionen numerisch **differenziert** oder **integriert**.  
 * Wie man die eingelesenen Daten mit Hilfe von *matplotlib* **darstellt**.
 * Wir demonstrieren hierzu die Berechnung der durch die Hystereseschleife eingeschlossenen Fläche mit der folgende Strategie: 
   * Die Datenpunkte $(t, U_{H})$ werden mit Hilfe einer *Spline*-Funktion **interpoliert**.
   * Mit Hilfe der Ableitung dieser *Spline*-Funktion lassen sich die **Zweige für mit $t$ zu- und abnehmende Werte von $U_{H}$**, für die spätere Integration, in die Datenreihen $U_{H}^{(+)}$ (`UHp`)und $U_{H}^{(-)}$ (`UHm`) aufspalten.
   * Die Datenpunkte $(U_{H}^{(+)}, U_{B})$ (unterer Teil der Hystereseschleife) und $(U_{H}^{(-)}, U_{B})$ (oberer Teil der Hystereseschleife) werden wieder mit Hilfe von *Spline*-Funktionen **interpoliert**.
   * Das **Integral des unteren Zweigs (`UHp`) wird schließlich vom Integral des oberen Zweigs (`UHm`) abgezogen**.

Alle im Folgenden gezeigten Code-Fragmente lassen sich geeignet kombinieren. 

## Einlesen der Daten im *csv*-Format mit *pandas*

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit *pandas* Daten aus einer *csv*-Datei aus dem PicoScope **einließt**.
 * Wir verwenden hierzu die Beispiel-Datei `Hysterese.csv`, aus dem *tools*-Verzeichnis.

In [16]:
import pandas as pd

# Read cvs file as pandas dataframe
df = pd.read_csv("Hysterese.csv")
# Translate dataframe columns into native python lists
t  = df["Frequency"].to_list()[1:]
UB = df["Channel A"].to_list()[1:] 
UH = df["Channel B"].to_list()[1:] 

## Einlesen der Daten im *csv*-Format mit PhyPraKit

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit der PhyPrakit Funktion *readPicoScope* Daten aus einer *csv*-Datei aus dem PicoScope **einließt**.
 * Wir verwenden hierzu die Beispiel-Datei `Hysterese.csv`, aus dem *tools*-Verzeichnis.

In [22]:
from PhyPraKit.PhyPraKit.phyTools import readPicoScope

# Read data from PicoScope; 
# * "data" is a list of lists which correspond to the columns in the csv file; 
# * "units" is the list of units per column; 
# check help(readPicoScope) to learn more about this function
units, data = readPicoScope("Hysterese.csv", delim=",")
# The association of data[1] and data[2] depends on the cabling of the PicoScope!
t  = data[0]  # column 0 in csv file: sampling time (not really needed)
UB = data[1]  # column 1 in csv file: proportional to B-Field
UH = data[2]  # column 2 in csv file: proportional to H-Field

## Smoothing und down sampling mit PhyPraKit

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit der PhyPrakit Funktion *meanFilter* eine Datenreihe durch laufende Mittelwertbildung (*running mean*) über $n$ Nachbarn **glättet**.
 * Wie man eine Datenreihe mit der PhyPraKit Funktion *resample* durch Mittelwertbildung über $n$ Nachbarn reduziert (**down sampling**).

In [ ]:
from PhyPraKit.phyTools import resample, meanFilter

# If length is too large, resample by an appropriate factor, we are fine with 
# 150 data points
if il > 150:
    n = int(il/150)
    # This is an example of smoothing by averaging over n neighbors
    #print("Smoothing with window size ", n)
    #t  = meanFilter(vUH, width=n)
    #UH = meanFilter(vUH, width=n)
    #UB = meanFilter(vUB, width=n)
    # This is an example of down sampling by averaging over n neighbors
    print("Resampling by factor", n)
    t  = resample(t , n=n)
    UH = resample(UH, n=n)
    UB = resample(UB, n=n)

## Interpolation durch *Spline*-Funktionen und Darstellung mit *matplotlib*

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit der scipy Klasse [interpolate.UnivariateSpline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.UnivariateSpline.html#scipy.interpolate.UnivariateSpline) eine (beliebige) Datenreihe druch [*Spline*](https://de.wikipedia.org/wiki/Spline)-Funktionen **interpoliert**.
 * Wie man die Datenreihe und die daraus gewonnenen *Spline*-Funktionen für der Hystereseschleife mit den Methoden von *matplotlib* darstellt.

In [ ]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

# In a real example you have to calibrate your data e.g. at this point; we add 
# some fake calibration constants for demonstration purposes, here
CALIB_UH2H = 200.0  # U_H -> H <-- adjust !
CALIB_UB2B = 1.3    # U_B -> B <-- adjust !
H = UH * CALIB_UH2H
B = UB * CALIB_UB2B
# Interpolate the points of (t,H) by spline functions; s=0 means that no extra 
# smoothing will be applied, each point of H will be used for the spline
spl_Ht = interpolate.UnivariateSpline(t, H, s=0)
spl_Bt = interpolate.UnivariateSpline(t, B, s=0)

# Plot hysteresis curve as Channel A vs. Channeel B
tplt = np.linspace(t[0], t[-1], 200)
unitH = "(A/m)"; unitB = "(T)"
fig = plt.figure(1, figsize=(6.0, 6.0))
ax1 = fig.add_subplot()
ax1.scatter(H, B, color="blue", marker="o", s=15.0, label="Data points")
ax1.plot(spl_Ht(tplt), spl_Bt(tplt), color="darkblue", label="Spline function")
ax1.set_xlabel("H uncalib. " + unitH)
ax1.set_ylabel("B uncalib. " + unitB)
ax1.legend(numpoints=1, loc="best")
ax1.grid(linestyle="dashed")
plt.show()

## Differentiation und Integration von *Spline*-Funktionen

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit der scipy Klasse [interpolate.UnivariateSpline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.UnivariateSpline.html#scipy.interpolate.UnivariateSpline) eine (beliebige) Datenreihe druch [*Spline*](https://de.wikipedia.org/wiki/Spline)-Funktionen **interpoliert**.
 * Wie man nach der Interpolation die *Spline*-Funktion mit der Funktion [derive](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.UnivariateSpline.derivative.html#scipy.interpolate.UnivariateSpline.derivative) **numerisch ableitet**.
 * Wie man nach der Interpolation die *Spline*-Funktion mit der Funktion [integrate](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.UnivariateSpline.integral.html#scipy.interpolate.UnivariateSpline.integral) **numerisch integriert**.
 * Wir folgen dabei der in Zelle 2 dargelegten Strategie.

In [ ]:
# The derivatives of H(t) will be used to determine the upper and lower branches 
# of the hysteresis slope
spl_Ht_deriv = spl_Ht.derivative()
Hp = []; Bp = []; Hm = []; Bm = []
for i, ti in enumerate(t):
    if spl_Ht_deriv(ti) > 0.0:
        # The branch when H was increased (lower branch)
        Hp.append(H[i])
        Bp.append(B[i])
    else:
        # The branch when H was decreased (upper branch)
        Hm.append(H[i])
        Bm.append(B[i])
# Convert to numpy arrays and sort at the same time increasing in H
ip = np.argsort(Hp)
Hp = np.array(Hp)[ip]
Bp = np.array(Bp)[ip]
im = np.argsort(Hm)
Hm = np.array(Hm)[im]
Bm = np.array(Bm)[im]
# Provide spline approximations for the upper and lower bracnches of the curve; 
# adjusting the parameter s is a cruical ingredient here to get useful and 
# stable interpolations
prec = (max(Hp) - min(Hp))*10e-7
spl_BHp = interpolate.UnivariateSpline(Hp, Bp, s=prec)
spl_BHm = interpolate.UnivariateSpline(Hm, Bm, s=prec)
# Detemine boundaries of the contour for integration; choose the smallest 
# interval out the upper and lower branch of the curve
Hmin = max(np.min(Hp), np.min(Hm))
Hmax = min(np.max(Hp), np.max(Hm))
# Calculate the integral of the contour
integral = spl_BHm.integral(Hmin, Hmax) - spl_BHp.integral(Hmin, Hmax)
# Print the result to screen
print("Area enclosed by slope:", integral)

# Plot hysteresis curve as Channel A vs. Channeel B and highlight enclosed 
# area
fig = plt.figure(1, figsize=(6.0, 6.0))
ax2 = fig.add_subplot()
ax2.scatter(Hp, Bp, color="red", marker="o", s=15.0, label="Increasing H")
ax2.scatter(Hm, Bm, color="blue", marker="o", s=15.0, label="Decreasing H")
Hplt = np.linspace(Hmin, Hmax, 200)
ax2.plot(Hplt, spl_BHp(Hplt), color="darkred", label="Lower Spline")
ax2.plot(Hplt, spl_BHm(Hplt), color="darkblue", label="Upper Spline")
# Form a single contour to plot a filled area
ax2.fill(
    np.concatenate((Hplt, np.flipud(Hplt))),
    np.concatenate((spl_BHp(Hplt), np.flipud(spl_BHm(Hplt)))),
    color="blue",
    alpha=0.25, 
    label="Enclosed area"
)
ax2.legend(numpoints=1, loc="best")
ax2.set_xlabel("H uncalib. " + unitH)
ax2.set_ylabel("B uncalib. " + unitB)
ax2.grid(linestyle="dashed")
ax2.text(-275.0, -0.025, r"$\oint B\,\mathrm{d}H\,=\,%.4g\,\mathrm{\frac{J}{m^{3}}}$" % integral)
plt.show()

## Check

Führen Sie zur Wiederholung des Gelernten und zum Test Ihres Verständnisses noch einmal die folgenden Operationen durch:

 * Stellen Sie die [differenzielle Permeabilität](https://de.wikipedia.org/wiki/Magnetische_Permeabilit%C3%A4t#Abh%C3%A4ngigkeit_von_der_Feldst%C3%A4rke:_Differentielle_Permeabilit%C3%A4t) $\mu_{D}(H)$ graphish dar

In [ ]:
# ADD YOUR CODE BELOW